In [ ]:
import modin.pandas as pd
import nums
import nums.numpy as nps
nums.init()

# Preparation

### Load and preprocess dataset with Modin.

In [ ]:
%%time
higgs_train = pd.read_csv("training.zip")
higgs_train.loc[higgs_train['Label'] == 'b', 'Label'] = 0
higgs_train.loc[higgs_train['Label'] == 's', 'Label'] = 1
higgs_train = higgs_train.drop(columns=['EventId'])
columns = higgs_train.columns.values
X_columns, y_column = columns[:-1], columns[-1:]

### Convert Modin DataFrame to NumS BlockArray.

In [ ]:
%%time
X_train = nums.from_modin(higgs_train[X_columns].astype(float))
weights = X_train[:, -1]
X_train = X_train[:, :-1]
# Drop weight column from names.
X_columns = X_columns[:-1]
y_train = nums.from_modin(higgs_train[y_column].astype(int)).reshape(-1)

# Exploration

### Compute principal components of dataset.

In [ ]:
%%time
# Compute PCA via SVD.
C = nps.cov(X_train, rowvar=False)
V, S, VT = nps.linalg.svd(C)
assert nps.allclose(V, VT.T)
pc = X_train @ V

### Compute eigen values from singular values, and explained variance from eigen values.

In [ ]:
eigen_vals = S**2 / (X_train.shape[0] - 1)
explained_variance = eigen_vals / nps.sum(eigen_vals)
for i, val in enumerate(nps.cumsum(explained_variance).get()[:10]):
    print(i, val)

### Order features by explained variance.

In [ ]:
components = VT
sorted_variance = nps.argsort(-nps.sum(nps.abs(components[:2]), axis=0)).get()
for col in X_columns[sorted_variance]:
    print(col)

# Modelling

### Import scikit-learn models from nums.

In [ ]:
from nums.sklearn import (train_test_split, 
                          StandardScaler, 
                          GaussianNB, 
                          LogisticRegression, 
                          SVC, 
                          MLPClassifier, 
                          GradientBoostingClassifier, 
                          RandomForestClassifier)
print("Models imported.")

### Define the performance metric.

In [ ]:
def metric(ytrue, ypred, weights):
    """ Approximate Median Significance defined as:
        AMS = sqrt(
                2 { (s + b + b_r) log[1 + (s/(b+b_r))] - s}
              )
    where b_r = 10, b = background, s = signal, log is natural logarithm """
    # True-positive rate.
    s = nps.sum(weights[(ytrue == 1) & (ytrue == ypred)])
    # False-positive rate.
    b = nps.sum(weights[(ytrue == 1) & (ytrue != ypred)])
    br = 10.0
    radicand = 2 * ((s + b + br) * nps.log(1.0 + s / (b + br)) - s)
    return nps.sqrt(radicand)
print("Metric defined.")

### Conduct a search over a small set of feature sets, preprocessors, and models.

In [ ]:
%%time
scores = []
for drop_features in [0, 3]:
    if drop_features > 0:
        feature_mask = nps.zeros(shape=sorted_variance.shape, dtype=bool)
        feature_mask[sorted_variance[:-drop_features]] = True
        Xt, Xv, yt, yv, wt, wv = train_test_split(X_train[:, feature_mask], y_train, weights)
    else:
        Xt, Xv, yt, yv, wt, wv = train_test_split(X_train, y_train, weights)
    print("drop_features", drop_features)
    numfeatstr = "num_feats=%s" % Xt.shape[1]
    for p_cls in [StandardScaler, None]:
        if p_cls is None:
            ppstr = "preproc=None"
            pXt = Xt
            pXv = Xv
        else:
            ppstr = "preproc=" + p_cls.__name__
            p_inst = p_cls()
            pXt = p_inst.fit_transform(Xt)
            pXv = p_inst.fit_transform(Xv)

        # Tree-based Ensemble Methods
        for n_estimators in [10]:
            for max_depth in [2]:
                for max_features in [None]:
                    m = RandomForestClassifier(n_estimators=n_estimators, 
                                               max_depth=max_depth, max_features=max_features)
                    m.fit(pXt, yt)
                    scores.append([(numfeatstr + ", "
                                    + ppstr + ", "
                                    + m.__class__.__name__
                                    + ("(%s, %s, %s)" % (n_estimators, max_depth, max_features))),
                                   (m.predict(pXv), yv, wv)])
                    for learning_rate in [.4]:
                        for subsample in [.9]:
                            m = GradientBoostingClassifier(n_estimators=n_estimators, 
                                                           max_depth=max_depth, 
                                                           max_features=max_features,
                                                           learning_rate=learning_rate,
                                                           subsample=subsample)
                            m.fit(pXt, yt)
                            scores.append([(numfeatstr + ", "
                                            + ppstr + ", "
                                            + m.__class__.__name__ 
                                            + ("(%s, %s, %s, %s, %s)" % (n_estimators, 
                                                                max_depth, 
                                                                max_features,
                                                                learning_rate,
                                                                subsample))),
                                           (m.predict(pXv), yv, wv)])
print("Training %s pipelines." % len(scores))

### Run performance metric and sort the pipelines by their performance.

In [ ]:
%%time
results = []
for res in scores:
    results.append((res[0], metric(*res[1]).get()))
for res in sorted(results, key=lambda x: -x[-1]):
    print(*res)

# Sources
- https://www.kaggle.com/c/higgs-boson/code
- https://nycdatascience.com/blog/student-works/top2p-higgs-boson-machine-learning/